In [7]:
import numpy as np

def autocorrelation(x, lag=1):
    """
    Compute the autocorrelation for a given lag in an MCMC sample.

    Parameters:
    x (numpy.ndarray): The MCMC sample array.
    lag (int): The lag at which to compute the autocorrelation.

    Returns:
    numpy.ndarray: The autocorrelation coefficients for each dimension.
    """
    n, d = x.shape
    mean = np.mean(x, axis=0)
    var = np.var(x, axis=0)

    autocorr = np.zeros(d)
    for dim in range(d):
        total = 0.0
        for i in range(n - lag):
            total += (x[i, dim] - mean[dim]) * (x[i + lag, dim] - mean[dim])
        autocorr[dim] = total / ((n - lag) * var[dim])

    return autocorr

def simulate_mcmc(n_samples=1000, n_dim=2, step_size=0.1):
    """
    Simulate a Markov Chain Monte Carlo (MCMC) sample using a simple random walk.

    Parameters:
    n_samples (int): Number of samples to generate.
    n_dim (int): Number of dimensions.
    step_size (float): The step size of the random walk.

    Returns:
    numpy.ndarray: The generated MCMC sample.
    """
    current_position = np.zeros(n_dim)
    samples = np.zeros((n_samples, n_dim))

    for i in range(n_samples):
        step = np.random.normal(0, step_size, n_dim)
        current_position += step
        samples[i] = current_position

    return samples

# Simulate MCMC samples
n_samples = 10_000_000
n_dim = 10  # number of dimensions
mcmc_sample = simulate_mcmc(n_samples, n_dim)

# Compute the autocorrelation
autocorr = autocorrelation(mcmc_sample, lag=100000)
print("Autocorrelation at lag 1 for each dimension:", autocorr)

Autocorrelation at lag 1 for each dimension: [0.94266755 0.97565173 0.94491125 0.96199394 0.95560293 0.97198032
 0.95855234 0.91952298 0.97663201 0.92934786]


<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [9]:
autocorr = autocorrelation(mcmc_sample, lag=1_000_000)
print("Autocorrelation at lag 1 for each dimension:", autocorr)

Autocorrelation at lag 1 for each dimension: [0.30382938 0.71930478 0.48516302 0.73338344 0.69587005 0.70535785
 0.49628092 0.26966161 0.83905799 0.51008626]


In [16]:
import numpy as np
import pandas as pd

def compute_multivariate_autocorrelation(data, lags):
    """
    Compute the autocorrelation for a multivariate time series.

    Parameters:
    data (numpy.ndarray): Multivariate time series data.
    lags (int): Number of lags to compute autocorrelation for.

    Returns:
    dict: A dictionary containing autocorrelation matrices for each lag.
    """
    n, m = data.shape
    autocorr_matrices = {}

    # Normalize data
    data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)

    for lag in range(lags + 1):
        # Initialize autocorrelation matrix for this lag
        autocorr_matrix = np.zeros((m, m))

        for i in range(m):
            for j in range(m):
                if lag == 0:
                    # Autocorrelation at lag 0 is the correlation coefficient
                    autocorr_matrix[i, j] = np.corrcoef(data[:, i], data[:, j])[0, 1]
                else:
                    # Compute autocorrelation for lag > 0
                    autocorr_matrix[i, j] = np.corrcoef(data[:-lag, i], data[lag:, j])[0, 1]

        autocorr_matrices[lag] = autocorr_matrix

    return autocorr_matrices

# Example usage
np.random.seed(0)
n_samples = 1000000
n_variables = 3

# Simulate some multivariate time series data
data = np.random.rand(n_samples, n_variables)

# Compute autocorrelation up to lag 5
autocorr_results = compute_multivariate_autocorrelation(data, 5)

# Display autocorrelation matrices for each lag
for lag, matrix in autocorr_results.items():
    print(f"Autocorrelation matrix for lag {lag}:\n", pd.DataFrame(matrix), "\n")

Autocorrelation matrix for lag 0:
           0         1         2
0  1.000000  0.000426 -0.000182
1  0.000426  1.000000  0.000294
2 -0.000182  0.000294  1.000000 

Autocorrelation matrix for lag 1:
           0         1         2
0 -0.001797  0.000196 -0.002158
1 -0.000549 -0.000962 -0.000757
2 -0.002081  0.000216 -0.001676 

Autocorrelation matrix for lag 2:
           0         1         2
0  0.000842  0.000156 -0.000002
1  0.000457 -0.001347  0.000356
2  0.000163  0.000926 -0.000732 

Autocorrelation matrix for lag 3:
           0         1         2
0 -0.000612 -0.000120 -0.000441
1 -0.001765 -0.000932 -0.000989
2  0.000204 -0.001208 -0.001830 

Autocorrelation matrix for lag 4:
           0         1         2
0  0.000288 -0.000528 -0.000050
1  0.001096 -0.000957  0.000913
2  0.001303  0.000614 -0.000026 

Autocorrelation matrix for lag 5:
           0         1         2
0  0.000543 -0.000212  0.000850
1 -0.000699 -0.000335 -0.000079
2  0.000108 -0.001569  0.000493 



In [17]:
from scipy.stats import pearsonr

def test_autocorrelation_significance(data, autocorr_matrices, alpha=0.05):
    """
    Test the statistical significance of each autocorrelation.

    Parameters:
    data (numpy.ndarray): Multivariate time series data.
    autocorr_matrices (dict): Dictionary of autocorrelation matrices for each lag.
    alpha (float): Significance level.

    Returns:
    dict: A dictionary containing p-values for each autocorrelation coefficient.
    """
    n, m = data.shape
    p_values = {lag: np.zeros_like(matrix) for lag, matrix in autocorr_matrices.items()}

    for lag, matrix in autocorr_matrices.items():
        for i in range(m):
            for j in range(m):
                if lag == 0:
                    # At lag 0, use Pearson correlation test
                    corr, p = pearsonr(data[:, i], data[:, j])
                else:
                    # For lag > 0, use time-shifted data
                    corr, p = pearsonr(data[:-lag, i], data[lag:, j])
                p_values[lag][i, j] = p

    return p_values

# Test the statistical significance of the autocorrelations
p_values = test_autocorrelation_significance(data, autocorr_results)

# Display p-values for each lag
for lag, matrix in p_values.items():
    print(f"P-values for autocorrelation at lag {lag}:\n", pd.DataFrame(matrix), "\n")

P-values for autocorrelation at lag 0:
           0         1         2
0  0.000000  0.670154  0.855195
1  0.670154  0.000000  0.768557
2  0.855195  0.768557  0.000000 

P-values for autocorrelation at lag 1:
           0         1         2
0  0.072265  0.844890  0.030943
1  0.583192  0.336054  0.448774
2  0.037454  0.829011  0.093767 

P-values for autocorrelation at lag 2:
           0         1         2
0  0.399734  0.876292  0.998371
1  0.648030  0.178105  0.721845
2  0.870845  0.354576  0.464435 

P-values for autocorrelation at lag 3:
           0         1         2
0  0.540405  0.904690  0.658869
1  0.077524  0.351258  0.322795
2  0.838354  0.227060  0.067222 

P-values for autocorrelation at lag 4:
           0         1         2
0  0.773528  0.597769  0.960015
1  0.272901  0.338514  0.361507
2  0.192453  0.539486  0.979199 

P-values for autocorrelation at lag 5:
           0         1         2
0  0.587161  0.832151  0.395437
1  0.484560  0.737896  0.936997
2  0.913637  0